In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# inputs
thickness = 220e-9
mode = 'TM'

## Effectives Indicies Function
Finds the effective indicies of a 3 layer slab waveguide

In [4]:
def TE_eq(b0,k0,n1,n2,n3,t):
    h0 = np.sqrt( (n2*k0)^2 - b0**2 )
    q0 = np.sqrt( b0**2 - (n1*k0)**2 )
    p0 = np.sqrt( b0**2 - (n3*k0)**2 )
    # the objective is to find zeroes of te0 and tm0
    te0 = np.tan( h0*t ) - (p0+q0)/h0/(1-p0*q0/h0**2)
    return te0, h0, q0, p0

In [5]:
def TM_eq(b0,k0,n1,n2,n3,t):
    h0 = np.sqrt( (n2*k0)^2 - b0**2 )
    q0 = np.sqrt( b0**2 - (n1*k0)^2 )
    p0 = np.sqrt( b0**2 - (n3*k0)^2 )
    pbar0 = (n2/n3)**2*p0
    qbar0 = (n2/n1)**2*q0
    tm0 = np.tan( h0*t ) - h0*(pbar0+qbar0)/(h0**2-pbar0*qbar0)
    return tm0,h0,q0,p0

In [7]:
# This functions finds the analtical solutions to the effectives indicies for the TM and TE modes.
def wg_1D_analytic(wavelength, t, n1, n2, n3):
    k0 = 2*np.pi/wavelength # the wave vector
    b0 = np.linspace(max(n1,n3)*k0, n2*k0, 1000)   #k0*n3 less than b less than k0*n2
    b0 = b0[1:-1]
    te0=TE_eq(b0,k0,n1,n2,n3,t)
    tm0=TM_eq(b0,k0,n1,n2,n3,t)

## Mode Profile Function
Finds the mode profiles (E and H components) for the TE and TM modes

In [ ]:
TE_E = list()

def wg_1D_mode_profile(wavelength, t, n1, n2, n3, pts, M):
    nTE,nTM,TEparam,TMparam= wg_1D_analytic(wavelength,t,n1,n2,n3)

    x1=np.linspace( -M*t, -t/2, pts)
    x2=np.linspace( -t/2, t/2, pts) 
    x3=np.linspace( t/2, M*t, pts)
    x = [x1, x2, x3]
    nx = [n1*np.matlib.ones(pts,1), n2*np.matlib.ones(pts,1), n3*np.matlib.ones(pts,1)]

    mu0=4*np.pi*1e-7
    epsilon0=8.85e-12
    eta=np.sqrt(mu0/epsilon0)
    c=3e8
    
    for i in range(nTE):
        h = TEparam(i,2)
        q = TEparam(i,3)
        p = TEparam(i,4)
        beta = 2*np.pi*nTE(i)/wavelength;
		C = 2*h*np.sqrt( 2*np.pi*c/wavelength*mu0 / (beta * (t+1/q+1/p)*(h**2+q**2) ) ) # normalize to 1W
		# n1, n2, n3 regions
		TE_E.append(C*(np.exp(q*(x1+t/2)), (np.cos(h*(x2+t/2))+q/h*np.sin(h*(x2+t/2))), (np.cos(h*t)+q/h*np.sin(h*t))*np.exp(-p*(x3-t/2))))


## Main Function

In [ ]:
x, TE_E, TE_H, TM_E, TM_H, nTE, nTM = wg_1D_mode_profile(1.55e-6, thickness, 1.444, 3.47, 1.444, 100, 3)